In [ ]:
import albumentations as alb
import tensorflow as tf
import os
import cv2 as cv
import numpy as np
import json
import matplotlib.pyplot as plt

In [ ]:
augmentation = alb.Compose([alb.RandomCrop(width=450,height=450),
                            alb.HorizontalFlip(p=0.5),
                            alb.RandomBrightnessContrast(p=0.2),
                            alb.RandomGamma(p=0.2),
                            alb.RGBShift(p=0.2),
                            alb.VerticalFlip(p=0.5),
                            alb.VerticalFlip(p=0.5)],
                            bbox_params=alb.BboxParams(format='albumentations',
                                                       label_fields=['class_labels']))

In [ ]:
img = cv.imread("C:\\Users\\sanji\\Desktop\\OBJECT_DETECTION\\data\\Train\\images\\5a588da4-be64-11ed-9c24-c49dacf3fc5b.jpg")

In [ ]:
with open("C:\\Users\\sanji\\Desktop\\OBJECT_DETECTION\\data\\Train\\labels\\5a588da4-be64-11ed-9c24-c49dacf3fc5b.json") as f:
    i = json.load(f)

In [ ]:
i

In [ ]:
coord = [0,0,0,0]
coord[0] = i['shapes'][0]['points'][0][0]
coord[1] = i['shapes'][0]['points'][0][1]
coord[2] = i['shapes'][0]['points'][1][0]
coord[3] = i['shapes'][0]['points'][1][1]

In [ ]:
coord

In [ ]:
coords = list(np.divide(coord,[640,480,640,480]))

In [ ]:
coords

In [ ]:
augmented = augmentation(image = img,bboxes=[coords],class_labels=['Face'])

In [ ]:
augmented

In [ ]:
cv.rectangle(augmented['image'],tuple(np.multiply(augmented['bboxes'][0][:2],[450,450]).astype(int)),
             tuple(np.multiply(augmented['bboxes'][0][2:],[450,450]).astype(int)),
             (0,255,0),2)
plt.imshow(augmented['image'])

In [ ]:
dirs = ['Train','Test','Val']
for i in dirs:
    for img in os.listdir(os.path.join('data',i,'images')):
        image = cv.imread(os.path.join('data',i,'images',img))
        coords = [0,0,0.00001,0.00001]
        label_path = ".\\data\\{}\\lables\\{}".format(i,img.split(".")[0]+".json")
        if os.path.exists(label_path):
            with open(label_path,'r') as f:
                label = json.load(f)
            coord = [0,0,0,0]
            coord[0] = label['shapes'][0]['points'][0][0]
            coord[1] = label['shapes'][0]['points'][0][1]
            coord[2] = label['shapes'][0]['points'][1][0]
            coord[3] = label['shapes'][0]['points'][1][1]
            coord = list(np.divide(coords,[640,480,640,480]))
        try:
            for k in range(60):
                augmented = augmentation(image=image,bboxes=[coord],class_labels = ['Face'])
                cv.imwrite(os.path.join('augm_data',i,'images',f'{img.split(".")[0]}.{k}.jpg'),augmented['image'])

                annotation = {}
                annotation['image'] = img

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0:
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0
                    else:
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else:
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0
                with open(os.path.join('augm_data',i,'labels',f'{img.split(".")[0]}.{k}.json'),'w') as f:
                    json.dump(annotation,f)
        except Exception as e:
            print(e)